In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:
    y = tf.square(x)
y_grad = tape.gradient(y,x)
print(y,y_grad)

tf.Tensor(9.0, shape=(), dtype=float32) tf.Tensor(6.0, shape=(), dtype=float32)


In [3]:
X = tf.constant([[1., 2.], [3., 4.]])
y = tf.constant([[1.], [2.]])
w = tf.Variable(initial_value=[[1.], [2.]])
b = tf.Variable(initial_value=1.)
with tf.GradientTape() as tape:
    L = tf.reduce_sum(tf.square(tf.matmul(X,w) + b - y))
w_grad, b_grad = tape.gradient(L, [w, b])        # 計算L(w, b)關於w, b的偏導數
print(L, w_grad, b_grad) 

tf.Tensor(125.0, shape=(), dtype=float32) tf.Tensor(
[[ 70.]
 [100.]], shape=(2, 1), dtype=float32) tf.Tensor(30.0, shape=(), dtype=float32)


# 考慮一個實際問題，某城市在 2013 年 - 2017 年的房價如下表所示：

## 年份 2013   2014   2015   2016   2017

## 房價 12000 14000 15000 16500 17500

In [11]:
import numpy  as np
X_raw = np.array([2013, 2014, 2015, 2016, 2017], dtype=np.float32)
y_raw = np.array([12000, 14000, 15000, 16500, 17500], dtype=np.float32)

# 正規化處理
X = (X_raw - X_raw.min()) / (X_raw.max() - X_raw.min())
y = (y_raw - y_raw.min()) / (y_raw.max() - y_raw.min())
print(X)
print(y)

[0.   0.25 0.5  0.75 1.  ]
[0.         0.36363637 0.54545456 0.8181818  1.        ]


# Numpy下的線性回歸
機器學習模型的實現並不是 TensorFlow 的專利。事實上，對於簡單的模型，即使使用常規的科學計算套件或者工具也可以求解。在這裡，我們使用 NumPy 這一通用的科學計算套件來實現梯度下降方法。NumPy 提供了多維陣列支援，可以表示向量、矩陣以及更高維的張量。同時，也提供了大量支援在多維陣列上進行操作的函數（比如下面的 np.dot() 是求內積， np.sum() 是求和）。在這方面，NumPy 和 MATLAB 比較類似。在以下程式碼中，我們推導求損失函數關於參數 a 和 b 的偏導數，並使用梯度下降法反複疊代，最終獲得 a 和 b 的值

In [12]:
a, b = 0, 0

num_epoch = 10000
learning_rate = 5e-4
for e in range(num_epoch):
    # 手動計算損失函數關於自變數（模型參數）的梯度
    y_pred = a * X + b
    grad_a, grad_b = 2 * (y_pred - y).dot(X), 2 * (y_pred - y).sum()

    # 更新參數
    a, b = a - learning_rate * grad_a, b - learning_rate * grad_b

print(a, b)

0.9763702027872221 0.057564988311377796


# TensorFlow 下的線性回歸
TensorFlow 的 即時執行模式 與上述 NumPy 的運行方式十分類似，然而提供了更快速的運算（GPU 支援）、自動推導、優化器等一系列對深度學習非常重要的功能。以下展示了如何使用 TensorFlow 計算線性回歸。可以注意到，程式的結構和前述 NumPy 的實現非常類似。這裡，TensorFlow 幫助我們做了兩件重要的工作：

In [16]:
X = tf.constant(X)
y = tf.constant(y)

a = tf.Variable(initial_value=1.)
b = tf.Variable(initial_value=2.)
variables = [a, b]

num_epoch = 10000
optimizer = tf.keras.optimizers.SGD(learning_rate=5e-4)
for e in range(num_epoch):
    # 使用tf.GradientTape()記錄損失函數的梯度資訊
    with tf.GradientTape() as tape:
        y_pred = a*X + b
        loss = tf.reduce_sum(tf.square(y_pred-y))
    # TensorFlow自動計算損失函數關於自變數（模型參數）的梯度
    grads = tape.gradient(loss, variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, variables))
    
print(a,b)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.9757488> <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.057909347>
